In [13]:
import numpy as np
from sklearn import svm
from dprl.ldp import NumericAndCategorical, SGD

from sklearn import metrics
import dprl
import tensorflow as tf
from copy import deepcopy

from dprl.utils import process_train_data
from examples.data_utils import *
from sklearn.datasets import make_blobs

# DPRL Example

## 1 Discrete part

In [14]:
# dict_no = {
#     'train': 2000,
#     'test': 100
# }
# load_tabular_data('adult', dict_no, 'adult_train.csv', 'adult_test.csv', idx_sorted=True)

In [ ]:
avg_l = []
rl_l = []
random_l = []
greedy_l = []

for i in range(10):
    tf.reset_default_graph()

    total_epsilon = 3400
    num = 2000
    epsilon_level = [15, 1] # 24 * 10 + 36 * 5

    for _ in range(1):
        # load data
        x_train, y_train, x_test, y_test, col = preprocess_data('adult_train.csv', 'adult_test.csv')

        # Defines problem
        problem = 'classification'

        # Network parameters
        parameters = dict()
        parameters['activation'] = tf.nn.relu
        parameters['hidden_dim'] = 100
        parameters['comb_dim'] = 10
        parameters['iterations'] = 2000
        parameters['layer_number'] = 5
        parameters['batch_size'] = 2000
        parameters['learning_rate'] = 0.01
        parameters['epsilon_budget'] = total_epsilon
        parameters['epsilon_level'] = epsilon_level

        # Defines predictive model
        pred_model = svm.SVC(decision_function_shape='ovo')

        # Flags for using stochastic gradient descent / pre-trained model
        flags = {'sgd': False, 'pretrain': False}

        # Initializes DVRL
        dprl_class = dprl.Dprl(x_train, y_train, x_test, y_test,
                               problem, pred_model, parameters, 'None', flags)

        # Trains DVRL
        dprl_class.train_dprl('accuracy')

        print('Finished dprl training.')

        print(y_train)

        print(x_train)

        y_train = y_train
        # Estimates data values
        dve_out = dprl_class.data_valuator(x_train, y_train/2)

        # Predicts with DVRL
        print(dve_out)

        print('Finished data valuation.')

    a = np.asarray(dve_out)
    save_npy('value.npy', a)

    model = svm.SVC(decision_function_shape='ovo')

    dve_out = load_npy('value.npy')

    sorted_idx = np.argsort(dve_out)

    high_idx = sorted_idx[:100]
    low_idx = sorted_idx[100:]

    # load data
    x_train, y_train, x_test, y_test, col = preprocess_data(
        'adult_train.csv', 'adult_test.csv')

    epsilon_list = np.zeros(len(y_train))

    for i in high_idx:
        epsilon_list[i] = epsilon_level[0]

    for i in low_idx:
        epsilon_list[i] = epsilon_level[1]



    # process

    x_train_s, y_train_s = process_train_data(x_train,
                                              y_train,
                                              epsilon_list)

    model.fit(x_train_s, y_train_s)
    y_pred = model.predict(x_test)
    acc = metrics.accuracy_score(y_test, y_pred, normalize=True)

    print('acc rl: \t', acc)
    rl_l.append(acc)

    epsilon_list_n = [total_epsilon/len(y_train)] * len(y_train)

    x_train_n, y_train_n = process_train_data(x_train,
                                              y_train,
                                              epsilon_list)
    model.fit(x_train_n, y_train_n)
    y_pred = model.predict(x_test)
    acc = metrics.accuracy_score(y_test, y_pred, normalize=True)

    print('acc baseline: \t', acc)
    avg_l.append(acc)


    # load data
    x_train, y_train, x_test, y_test, col = preprocess_data(
        'adult_train.csv', 'adult_test.csv')

    epsilon_list_random = np.zeros(len(y_train))

    for i in range(100):
        epsilon_list_random[i] = epsilon_level[0]

    for i in range(100, 2000):
        epsilon_list_random[i] = epsilon_level[1]



    # process

    x_train_random, y_train_random = process_train_data(x_train,
                                              y_train,
                                              epsilon_list_random)

    model.fit(x_train_random, y_train_random)
    y_pred = model.predict(x_test)
    acc = metrics.accuracy_score(y_test, y_pred, normalize=True)

    print('acc random: \t', acc)
    random_l.append(acc)

    # def test():
    #     epsilon_list_g = [epsilon_level[1]] * 2000
    #     for kk in range(400):
    #         comp_list = []
    #         index_list = []
    #         select_index = 0
    #         for gi in range(400):
    #             if epsilon_list_g[gi] == epsilon_level[1]:
    #                 # try
    #                 temp_el = deepcopy(epsilon_list_g)
    #                 temp_el[gi] = epsilon_level[0]
    #                 temp_x, temp_y = process_train_data(x_train, y_train, temp_el)
    #                 model.fit(temp_x, temp_y)
    #                 y_pred = model.predict(x_test)
    #                 acc = metrics.accuracy_score(y_test, y_pred, normalize=True)
    #                 comp_list.append(acc)
    #                 index_list.append(gi)
    #
    #         v = 0
    #         for walk in range(len(comp_list)):
    #             if comp_list[walk] > v:
    #                 v = comp_list[walk]
    #                 select_index = walk
    #
    #         # select_index = np.argmax(comp_list, axis=0)
    #         ii = index_list[select_index]
    #         epsilon_list_g[ii] = epsilon_level[0]
    #         print(epsilon_list_g)
    #
    #     x_train_g, y_train_g = process_train_data(x_train,
    #                                                y_train,
    #                                                epsilon_list_g)
    #
    #     model.fit(x_train_g, y_train_g)
    #     y_pred = model.predict(x_test)
    #     acc = metrics.accuracy_score(y_test, y_pred, normalize=True)
    #     return acc
    #
    # greedy_l.append(test())







    # def eval_utility(x_train, y_train, x_test, y_test, model) -> float:
    #     """Evaluate the coalition utility.
    #     """
    #
    #     single_pred_label = (True if len(np.unique(y_train)) == 1
    #                          else False)
    #
    #     if single_pred_label:
    #         y_pred = [y_train[0]] * len(y_test)
    #     else:
    #         model.fit(x_train, y_train)
    #         y_pred = model.predict(x_test)
    #
    #     return metrics.accuracy_score(y_test, y_pred, normalize=True)

# save_npy('rl2.npy', rl_l)
# save_npy('random2.npy', random_l)
# save_npy('avg2.npy', avg_l)
# save_npy('greedy2.npy', greedy_l)


Finished dprl training.
[0 0 0 ... 0 0 0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
y_train_valid_pred [1. 0. 1. ... 0. 1. 0.]
INFO:tensorflow:Restoring parameters from None
[0.85433495 0.6636017  0.8299963  ... 0.9955667  0.50942904 0.5326499 ]
Finished data valuation.
acc rl: 	 0.75
acc baseline: 	 0.75
acc random: 	 0.75
Finished dprl training.
[0 0 0 ... 0 0 0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
y_train_valid_pred [1. 0. 1. ... 0. 1. 0.]
INFO:tensorflow:Restoring parameters from None
[0.4883787  0.46848267 0.13085485 ... 0.7440434  0.57074714 0.24031037]
Finished data valuation.
acc rl: 	 0.75
acc baseline: 	 0.75
acc random: 	 0.75
Finished dprl training.
[0 0 0 ... 0 0 0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ..

 40%|███▉      | 793/2000 [01:32<05:32,  3.64it/s]